In [1]:
import torch
import pandas as pd
from PIL import Image
import torchvision.transforms as transforms
from torchvision import datasets, models
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import warnings
import os
warnings.filterwarnings('ignore')
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
from torchvision.datasets import EuroSAT
from torch.utils.tensorboard import SummaryWriter
# %load_ext tensorboard
import datetime
import time

from torchvision.models.resnet import *
from torchvision.models.resnet import BasicBlock, Bottleneck



device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [2]:
os.getcwd()

'/home/jupyter/facebook-marketplaces-recommendation-ranking-system/Practicals'

In [3]:
## Creating a train dataset class
class TrainAntBeeDataSet(Dataset):
    def __init__(self):
        super().__init__()
        self.examples = self._load_examples()
        self.pil_to_tensor = transforms.ToTensor()
        self.resize = transforms.Resize((225,225))

    def _load_examples(self):
        class_names = os.listdir('hymenoptera_data/train')
        class_encoder = {class_name: idx for idx, class_name in enumerate(class_names)}
        class_decoder = {idx: class_name for idx, class_name in enumerate(class_names)}

        examples_list = []
        for cl_name in class_names:
            example_fp = os.listdir(os.path.join('hymenoptera_data/train',cl_name))
            example_fp = [os.path.join('hymenoptera_data/train', cl_name, img_name ) for img_name in example_fp]
            example = [(img_name, class_encoder[cl_name]) for img_name in example_fp]
            examples_list.extend(example)

        return examples_list

    def __getitem__(self, idx):
        img_fp, img_class = self.examples[idx]
        img = Image.open(img_fp)

        features = self.pil_to_tensor(img)
        features = self.resize(features)

        return features, img_class

    def __len__(self):
        return len(self.examples)

In [4]:
## Creates a validation dataset class
class ValidationAntBeeDataSet(Dataset):
    def __init__(self):
        super().__init__()
        self.examples = self._load_examples()
        self.pil_to_tensor = transforms.ToTensor()
        self.resize = transforms.Resize((225,225))

    def _load_examples(self):
        class_names = os.listdir('hymenoptera_data/val')
        class_encoder = {class_name: idx for idx, class_name in enumerate(class_names)}
        class_decoder = {idx: class_name for idx, class_name in enumerate(class_names)}
        examples_list = []
        
        for cl_name in class_names:
            example_fp = os.listdir(os.path.join('hymenoptera_data/val',cl_name))
            example_fp = [os.path.join('hymenoptera_data/val', cl_name, img_name ) for img_name in example_fp]
            example = [(img_name, class_encoder[cl_name]) for img_name in example_fp]
            examples_list.extend(example)

        return examples_list

    def __getitem__(self, idx):
        img_fp, img_class = self.examples[idx]
        img = Image.open(img_fp)

        features = self.pil_to_tensor(img)
        features = self.resize(features)

        return features, img_class

    def __len__(self):
        return len(self.examples)

In [5]:
dataset = TrainAntBeeDataSet()

In [6]:
len(dataset)

244

In [7]:
testval = ValidationAntBeeDataSet()
len(testval)

153

In [17]:
## Created a classifier based on the RESNET50 pretrained model

class AntBeeClassifier(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
        self.resnet50.fc = torch.nn.Linear(2048,2)
  
    def forward(self, X):
        return F.softmax(self.resnet50(X))

In [21]:
def train(model,traindataloader, valdataloader, epochs):
    optimiser = torch.optim.SGD(model.parameters(), lr=0.01)
    model_path = str(os.path.join('model_evaluation', time.strftime("%Y%m%d-%H%M%S")))   
    os.makedirs(model_path)
    os.makedirs(os.path.join(model_path, 'weights'))
    batch_idx = 0
    
    for epoch in range(epochs):
        training_loss = 0.0
        validation_loss = 0.0
        model.to(device)
        model.train()
        tr_num_correct = 0
        tr_num_examples = 0
        epoch_combo = 'epoch' + str(epoch)
        os.makedirs(os.path.join(model_path, 'weights', epoch_combo))
        for inputs, labels in traindataloader:
            #labels = labels.unsqueeze(1)
            #labels = labels.float()
            inputs = inputs.to(device)
            labels = labels.to(device)
            predictions = model(inputs)
            #print(predictions.shape)
            #print(labels.shape)
            loss = torch.nn.CrossEntropyLoss()
            loss = loss(predictions, labels)
            loss.backward()
            optimiser.step()
            model_save_dir = str(os.path.join(model_path, 'weights', epoch_combo, 'weights.pth'))
            full_path = str('/home/jupyter/facebook-marketplaces-recommendation-ranking-system/Practicals')
            #print(model_save_dir)
            #torch.save({'epoch': epoch,
            #    'model_state_dict': model.state_dict(),
            #    'optimizer_state_dict': optimiser.state_dict()}, 
                  #str(os.path.join(full_path, model_save_dir)))
            #           model_save_dir)
            torch.save({'epoch': epoch,
                  'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimiser.state_dict()},
                  str(os.path.join(full_path, model_save_dir)))

            optimiser.zero_grad()
            batch_idx += 1
            training_loss += loss.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(predictions, dim=1), dim=1)[1], labels)
            tr_num_correct += torch.sum(correct).item()
            tr_num_examples += correct.shape[0]
        training_loss /= len(traindataloader.dataset)

        model.eval()
        val_num_correct = 0
        val_num_examples = 0
        for inputs, labels in valdataloader:
            #labels = labels.unsqueeze(1)
            #labels = labels.float()
            inputs = inputs.to(device)
            labels = labels.to(device)
            predictions = model(inputs)
            loss = torch.nn.CrossEntropyLoss()
            loss = loss(predictions, labels)
            validation_loss += loss.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(predictions, dim =1), dim=1)[1], labels)
            val_num_correct += torch.sum(correct).item()
            val_num_examples += correct.shape[0]
        validation_loss /= len(valdataloader.dataset)        
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, train_accuracy = {:.2f},val_accuracy = {:.2f} '.format(epoch, training_loss, validation_loss, tr_num_correct / tr_num_examples,
                                                                                                                             val_num_correct / val_num_examples))

In [24]:
classifier = AntBeeClassifier()

Using cache found in /home/jupyter/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [25]:
## unfreeze last two layers
for param in classifier.resnet50.layer4.parameters():
  param.requires_grad=True

for param in classifier.resnet50.fc.parameters():
  param.requires_grad=True

AttributeError: 'ResNet' object has no attribute 'layer4'

In [45]:
classifier.resnet50.named_modules

<bound method Module.named_modules of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layers): Sequential(
    (0): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsamp

In [49]:
## accessing the module names

for name, module in classifier.resnet50.named_modules():
                      print(name)


conv1
bn1
relu
maxpool
layers
layers.0
layers.0.0
layers.0.0.conv1
layers.0.0.bn1
layers.0.0.conv2
layers.0.0.bn2
layers.0.0.conv3
layers.0.0.bn3
layers.0.0.relu
layers.0.0.downsample
layers.0.0.downsample.0
layers.0.0.downsample.1
layers.0.1
layers.0.1.conv1
layers.0.1.bn1
layers.0.1.conv2
layers.0.1.bn2
layers.0.1.conv3
layers.0.1.bn3
layers.0.1.relu
layers.0.2
layers.0.2.conv1
layers.0.2.bn1
layers.0.2.conv2
layers.0.2.bn2
layers.0.2.conv3
layers.0.2.bn3
layers.0.2.relu
layers.1
layers.1.0
layers.1.0.conv1
layers.1.0.bn1
layers.1.0.conv2
layers.1.0.bn2
layers.1.0.conv3
layers.1.0.bn3
layers.1.0.relu
layers.1.0.downsample
layers.1.0.downsample.0
layers.1.0.downsample.1
layers.1.1
layers.1.1.conv1
layers.1.1.bn1
layers.1.1.conv2
layers.1.1.bn2
layers.1.1.conv3
layers.1.1.bn3
layers.1.1.relu
layers.1.2
layers.1.2.conv1
layers.1.2.bn1
layers.1.2.conv2
layers.1.2.bn2
layers.1.2.conv3
layers.1.2.bn3
layers.1.2.relu
layers.1.3
layers.1.3.conv1
layers.1.3.bn1
layers.1.3.conv2
layers.1.3.bn

In [50]:
## define the layers to unfreeze and then retrain
layers_to_unfreeze = ['layers.2', 'layers.3']

for name, param in classifier.resnet50.named_parameters():
    for layer_name in layers_to_unfreeze:
        if layer_name in name:
            param.requires_grad = True
            break

In [51]:
train_dataset = TrainAntBeeDataSet()
val_dataset = ValidationAntBeeDataSet()
train_loader = DataLoader(dataset = train_dataset, batch_size=16)
val_loader = DataLoader(dataset = val_dataset, batch_size=16)
train(classifier, traindataloader= train_loader, valdataloader= val_loader, epochs=15)


Epoch: 0, Training Loss: 0.69, Validation Loss: 0.76, train_accuracy = 0.60,val_accuracy = 0.46 
Epoch: 1, Training Loss: 0.69, Validation Loss: 0.76, train_accuracy = 0.55,val_accuracy = 0.46 
Epoch: 2, Training Loss: 0.68, Validation Loss: 0.74, train_accuracy = 0.53,val_accuracy = 0.46 
Epoch: 3, Training Loss: 0.66, Validation Loss: 0.72, train_accuracy = 0.59,val_accuracy = 0.46 
Epoch: 4, Training Loss: 0.64, Validation Loss: 0.70, train_accuracy = 0.64,val_accuracy = 0.46 
Epoch: 5, Training Loss: 0.62, Validation Loss: 0.68, train_accuracy = 0.72,val_accuracy = 0.48 
Epoch: 6, Training Loss: 0.59, Validation Loss: 0.66, train_accuracy = 0.77,val_accuracy = 0.52 
Epoch: 7, Training Loss: 0.57, Validation Loss: 0.64, train_accuracy = 0.81,val_accuracy = 0.57 
Epoch: 8, Training Loss: 0.55, Validation Loss: 0.62, train_accuracy = 0.86,val_accuracy = 0.59 
Epoch: 9, Training Loss: 0.53, Validation Loss: 0.60, train_accuracy = 0.89,val_accuracy = 0.65 
Epoch: 10, Training Loss: 0.51

In [13]:
os.getcwd()

'/home/jupyter/facebook-marketplaces-recommendation-ranking-system/Practicals'